<p>TVP model's specification is as follows:</p>
<p>$$ y_t = \beta_{1t} x_{1t} + \beta_{2t} x_{2t} + ... + \beta_{kt} x_{kt} + e_t \\
        \beta_{it} = \beta_{i,t-1} + v_{it} \\
        e_t \sim N(0, \sigma^2) \\
        v_{it} \sim N(0, \sigma_i^2) $$</p>
<p>where $y_t$ is a process, $\beta_{it}$ are time varying parameters.</p>
<p>Presented code uses this model for modelling changing conditional variance of uncertainty in the U.S. monetary growth (chapter 3.4 of Kim and Nelson, 1999).</p>
<p>See also http://econ.korea.ac.kr/~cjkim/MARKOV/programs/tvp.opt - the same model implementation used to test this code against.</p>

In [1]:
"""
Kim and Nelson's (1989) time-varying-parameter model for modelling changing
conditional variance of uncertainty in the U.S. monetary growth (chapter
3.4 of Kim and Nelson, 1999).

Author: Valery Likhosherstov
License: Simplified-BSD
"""

import numpy as np
from statsmodels.tsa.statespace.tvp import TVPModel
from statsmodels.tsa.statespace.tests.results import results_tvp

dtype = np.float64

true = results_tvp.tvp
start = true['start']

# Model attributes
k_exog = 5

# Preparing endog and exog data

endog = np.array(true['data']['m1'], dtype=dtype)

exog = np.zeros((endog.shape[0], k_exog), dtype=dtype)

exog[:, 0] = 1
exog[:, 1] = true['data']['dint']
exog[:, 2] = true['data']['inf']
exog[:, 3] = true['data']['surpl']
exog[:, 4] = true['data']['m1lag']

# Instantiate the model
model = TVPModel(endog, exog=exog, dtype=dtype, alternate_timing=True,
                 loglikelihood_burn=start)

# Authors use the following initialization
model.initialize_known(np.zeros(k_exog, dtype=dtype),
        np.identity(k_exog, dtype=dtype) * 50)

# Fit the model from start parameters, used by authors
results = model.fit(np.array(np.array(true['untransformed_start_parameters']), dtype=dtype))

In [2]:
# Print results summary
print(results.summary())

                           Statespace Model Results                           
Dep. Variable:                      y   No. Observations:                  106
Model:                       TVPModel   Log Likelihood                 -97.092
Date:                Thu, 21 Jul 2016   AIC                            206.185
Time:                        18:51:58   BIC                            222.165
Sample:                             0   HQIC                           212.662
                                - 106                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
obs_var        0.1378      0.047      2.940      0.003       0.046       0.230
tvp_var0       0.0124      0.024      0.509      0.611      -0.035       0.060
tvp_var1       0.0003      0.001      0.395      0.6

In [1]:
%matplotlib inline

# Plot time varying parameters
results.plot_coefficient(variables=range(k_exog), figsize=(8,10))